In [1]:
!pip install parse
!pip install iribaker

In [ ]:
from rdflib import Graph, Dataset, URIRef, Literal, Namespace, RDF, XSD
from rdflib.namespace import OWL, RDF, RDFS
from iribaker import to_iri
import  pandas as pd
import logging
from parse import *

,sch_id,wd_id,sch_cname,wd_cname
0,http://schema.org/AccountingService,http://www.wikidata.org/wiki/Q23671509,AccountingService,Q23671509
1,http://schema.org/Action,http://www.wikidata.org/wiki/Q4026292,Action,Q4026292
2,http://schema.org/AdministrativeArea,http://www.wikidata.org/wiki/Q3455524,AdministrativeArea,Q3455524
3,http://schema.org/AdministrativeArea,http://www.wikidata.org/wiki/Q56061,AdministrativeArea,Q56061
4,http://schema.org/Airline,http://www.wikidata.org/wiki/Q46970,Airline,Q46970
...,...,...,...,...
312,http://schema.org/Winery,http://www.wikidata.org/wiki/Q156362,Winery,Q156362
313,http://schema.org/WorkersUnion,http://www.wikidata.org/wiki/Q178790,WorkersUnion,Q178790
314,http://schema.org/Zoo,http://www.wikidata.org/wiki/Q43501,Zoo,Q43501
315,http://schema.org/FALSE,http://www.wikidata.org/wiki/Q5432619,FALSE,Q5432619


In [4]:

from SPARQLWrapper import SPARQLWrapper, JSON
def getWDLabel(uri):
    sparq1=SPARQLWrapper("https://wikidata.demo.openlinksw.com/sparql")
    queryString= """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
        PREFIX wd: <http://www.wikidata.org/entity/> 
        SELECT  *
        WHERE {
        %s rdfs:label ?label .
        FILTER (langMatches( lang(?label), "EN" ) )
      } 
      limit 1
      """%(uri)
    sparq1.setQuery (queryString)
    sparq1.setReturnFormat(JSON)
    results=sparq1.query().convert()
    for r in results["results"]["bindings"]:
        return(r["label"]["value"])
    
    
def GetClassLabel(URIString):
    if len(URIString.split(':')) > 2:
        Label = URIString.split(':')[-1]
    else:
        Label = URIString.split('/')[-1]
    return Label
    



In [ ]:
#mapping.to_csv('wd_class_name_mapping.csv')
mapping=pd.read_csv('wd_class_name_mapping.csv',index_col=0)
mapping

,sch_id,wd_id,sch_cname,wd_cname
0,http://schema.org/AccountingService,http://www.wikidata.org/wiki/accounting_services,AccountingService,accounting_services
1,http://schema.org/Action,http://www.wikidata.org/wiki/action,Action,action
2,http://schema.org/Airline,http://www.wikidata.org/wiki/airline,Airline,airline
3,http://schema.org/Airport,http://www.wikidata.org/wiki/airport,Airport,airport
4,http://schema.org/AmusementPark,http://www.wikidata.org/wiki/amusement_park,AmusementPark,amusement_park
...,...,...,...,...
299,http://schema.org/Winery,http://www.wikidata.org/wiki/winery,Winery,winery
300,http://schema.org/WorkersUnion,http://www.wikidata.org/wiki/labor_union,WorkersUnion,labor_union
301,http://schema.org/Zoo,http://www.wikidata.org/wiki/zoo,Zoo,zoo
302,http://schema.org/FALSE,http://www.wikidata.org/wiki/false,FALSE,false


In [4]:
#reading dataset from csv
#fileName='YAGO_instances.csv'

fileName="WikiData_instances.csv"
df=pd.read_csv(fileName)

#A namespace for our resources
#data = 'https://yago-knowledge.org/resource/'
data='http://www.wikidata.org/wiki/resource/'


DATA = Namespace(data)

# A namespace for the schema (Classes)
#schema ='http://schema.org/'
schema='http://www.wikidata.org/wiki/'

CLASS = Namespace(schema)

# Creating a graph
graph = Graph()
graph.bind("owl", OWL)
graph.bind("rdfs", RDFS)

In [5]:
Del=['educational_organization', 'landform']
df=df[~df['Class_Name'].isin(Del)]
ClassList=[]
ClassList=mapping['wd_cname'].unique()
#ClassList=df['Class_Name'].unique()

In [ ]:
len(ClassList)

304

In [7]:
df=df.dropna()
a=mapping['wd_cname'].unique()
b=df['Class_Name'].unique()
set(b).difference(set(a))

set()

In [8]:
for i in ClassList:
    #adding classes to the graph
    classN = URIRef(to_iri(schema + str(i)))
    name = Literal(i, datatype=XSD['string']) #the class name label
    graph.add((classN, RDF.type, OWL.Class))
    #graph.add((classN, RDF.type, RDFS.Class))
    graph.add((classN, RDFS.label, name))

In [9]:
from random import sample

#df=df.groupby('Class_Name', as_index=False, group_keys=False).apply(lambda x: x.sample(min(500, len(x))))

for i in df.Class_Name.unique():
    MyList=list(df.loc[df['Class_Name'] == i, 'Instance_Name'])
    #if len(MyList)>50: this is used to create the samller version
     #   MyList=sample(MyList,50)    
    # adding instances of a class to the graph
    classN = URIRef(to_iri(schema + str(i)))
    for c in range(len(MyList)):
        try:
            MyList[c]= MyList[c].strip(' " ').replace(" ' ",'')
            instance = URIRef(to_iri(data + MyList[c]))
            graph.add((instance, RDF.type, classN ))
            instanceLabel=Literal(MyList[c], datatype=XSD['string']) #creating the label
            graph.add((instance, RDFS.label,instanceLabel))
        except AttributeError:
            pass

  

In [10]:
#outFile = 'TestCase/source.xml'
outFile= '/YWtestCase/target.rdf'
#'smallYWtestCase/target.xml'
with open(outFile, 'wb') as f:
    graph.serialize(f, format='xml')

In [13]:
classes=[]
for s, p, o in graph.triples((None, RDF.type, OWL.Class)):
    classes.append(str(s))
s=set(classes)
print(len(s))

304


In [ ]:
df.groupby('Class_Name').counts

AttributeError: 'DataFrameGroupBy' object has no attribute 'counts'

In [ ]:
df=df.groupby('Class_Name').apply(lambda x: x.sample(5000) if len(x)> 4000)


SyntaxError: invalid syntax (<ipython-input-51-6c0b360dbaf8>, line 1)